In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import stft
from PIL import Image
import os
import random
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False
# f = np.load(r"C:\Users\Timothy\Desktop\数学建模相关\二轮\数据集\acT1_cplx_stft_f.npy")
# t = np.load(r"C:\Users\Timothy\Desktop\数学建模相关\二轮\数据集\acT1_cplx_stft_t.npy")
# Zxx = np.load(r"C:\Users\Timothy\Desktop\数学建模相关\二轮\数据集\acT1_cplx_stft_Zxx.npy")
# acT1_complex = np.load(r"C:\Users\Timothy\Desktop\数学建模相关\二轮\数据集\data_amplitude_T1_complex.npy")
# df = pd.DataFrame(acT1_complex)
# Zxx100p_max_f = np.argmax(np.abs(Zxx[100:, :, :]), axis=1)


# print(f.shape, t.shape, Zxx.shape)
# print(Zxx100p_max_f.shape)

In [ ]:
Bone = np.array(f[Zxx100p_max_f])
y_train = np.zeros(320)
Bone_train = np.expand_dims(Bone[80:400], axis=-1)
Bone_pred = np.expand_dims(Bone, axis=-1)

# 创建images
# 遍历数据
for i in range(850):
    # 创建一个新的图像
    fig = plt.figure(figsize=(10, 6))
    plt.scatter(t, Bone[i], s=1)
    plt.ylim(-0.3, 0.3)
    # 保存图像到特定位置，名称为图像序号
    plt.savefig(f'C:/Users/Timothy/Desktop/数学建模相关/二轮/数据集/images/{i}.png')
    
    plt.close(fig)
# 创建spectrums
# 遍历数据
for i in range(850):
    # 创建一个新的图像
    fig = plt.figure(figsize=(10, 6))
    plt.specgram(df.iloc[i+100], NFFT=1024, Fs=1, noverlap=500, cmap='jet')
    plt.ylim(-0.3, 0.3)
    # 保存图像到特定位置，名称为图像序号
    plt.savefig(f'C:/Users/Timothy/Desktop/数学建模相关/二轮/数据集/spectrums/{i}.png')
    
    plt.close(fig)

In [31]:
# 列出所有的图片文件名
image_dir = 'C:/Users/Timothy/Desktop/数学建模相关/二轮/数据集/images/'
image_files = [f for f in os.listdir(image_dir) if f.endswith('.png')]

# 提取图片的序号
image_indices = [int(f[:-4]) for f in image_files]

# 选择序号为偶数且序号处于342-354或514-520的图片
target_indices = [i for i in image_indices if i % 2 ==0 and (342 <= i < 354 or 514 <= i <= 520)]
target_files = [f'{image_dir}{i}.png' for i in target_indices]

# 选择其余的图片
other_indices = [i for i in image_indices if i not in target_indices]
random.shuffle(other_indices)
other_files = [f'{image_dir}{i}.png' for i in other_indices[:5]]

# 创建标签
target_labels = [1] * len(target_files)
other_labels = [0] * len(other_files)

# 合并文件名和标签
image_files = target_files + other_files
train_labels = np.array(target_labels + other_labels)
# print(image_files, labels, len(image_files), len(labels))

# 读取图像文件并转换为 numpy 数组
train_images = np.array([np.array(Image.open(i)) for i in image_files])
# 将图像数据的范围从0-255转换为0-1
train_images = train_images / 255.0
# 创建测试数据
test_images = np.array([np.array(Image.open(f'{image_dir}{2*i+1}.png')) for i in range(425)])


In [22]:
print(train_images.shape, test_images.shape, train_labels.shape)

(20, 600, 1000, 4) (425, 600, 1000, 4) (20,)


In [32]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# 创建模型
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(600, 1000, 4)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(2, activation='softmax'))  # 假设你有两个类别：海杂波和目标

# 编译模型
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 训练模型
model.fit(train_images, train_labels, epochs=1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.3333 - loss: 0.7369


In [8]:
test_images2 = np.array([np.array(Image.open(
    f'C:/Users/Timothy/Desktop/数学建模相关/二轮/数据集/images/{i}.png')) for i in range(425)])

In [33]:
# 预测新的数据
predictions = model.predict(test_images)
print(np.argmax(predictions, axis=-1))

14/14 ━━━━━━━━━━━━━━━━━━━━ 11s 710ms/step
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
